In [31]:
from pymatgen.io.espresso.PWscf import PWxml
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.core import Spin
import xml.etree.ElementTree as ET
from monty.io import zopen
import xmltodict
import numpy as np

In [32]:
#filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_vcrelax.xml'
#filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_scf.xml'
filename = 'pymatgen/io/espresso/tests/data/Sr3PbO_bands.xml'
#filename = 'pymatgen/io/espresso/tests/data/FeO_DFTU.xml'
#filename = 'pymatgen/io/espresso/tests/data/MoS2_vdw.xml'
#filename = 'pymatgen/io/espresso/tests/data/Ni_spin_pol.xml'
pwout = PWxml(filename)

In [33]:
vasprun = Vasprun('pymatgen/io/espresso/tests/data/Sr3PbO_vasprun.xml')

In [34]:
with open(filename, encoding="UTF-8") as xml:
    data = xmltodict.parse(xml.read())['qes:espresso']

In [35]:
#data['output']['band_structure']['highestOccupiedLevel']

In [36]:
data['output']['band_structure'].get('lowestUnoccupiedLevel', None)

'3.537375162118481e-1'

In [37]:
#pwout.eigenvalue_band_properties

In [38]:
pwout.parameters['dft']

{'functional': 'PBE'}

In [39]:
print(pwout.run_type)
print(pwout.is_spin)
print(pwout.is_hubbard)
print(pwout.hubbards)
pwout.as_dict()

PBE
False
False
{}


{'pwscf_version': 7.0,
 'has_pwscf_completed': True,
 'nsites': 5,
 'unit_cell_formula': {'Sr': 3.0, 'Pb': 1.0, 'O': 1.0},
 'reduced_cell_formula': {'Sr': 3.0, 'Pb': 1.0, 'O': 1.0},
 'pretty_formula': 'Sr3PbO',
 'is_hubbard': False,
 'hubbards': {},
 'elements': ['O', 'Pb', 'Sr'],
 'nelements': 3,
 'run_type': 'PBE',
 'input': {'crystal': {'@module': 'pymatgen.core.structure',
   '@class': 'Structure',
   'charge': 0,
   'lattice': {'matrix': [[9.79395795815934, 0.0, 0.0],
     [0.0, 9.79395795815934, 0.0],
     [0.0, 0.0, 9.79395795815934]],
    'pbc': [True, True, True],
    'a': 9.79395795815934,
    'b': 9.79395795815934,
    'c': 9.79395795815934,
    'alpha': 90.0,
    'beta': 90.0,
    'gamma': 90.0,
    'volume': 939.4522399686228},
   'sites': [{'species': [{'element': 'Sr', 'occu': 1}],
     'abc': [0.4999999999999827, 0.0, 0.4999999999999827],
     'xyz': [4.8969789790795, 0.0, 4.8969789790795],
     'label': 'Sr',
     'properties': {}},
    {'species': [{'element': 'Sr', '

In [40]:
pwout.atomic_symbols

['O', 'Pb', 'Sr']